# Entraînement d'un réseau de neurones pour jouer au Go

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/auduvignac/deep_learning_go/blob/main/src/train_go_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

## Description

- [https://www.lamsade.dauphine.fr/~cazenave/DeepLearningProject.html](https://www.lamsade.dauphine.fr/~cazenave/DeepLearningProject.html)  
- L'objectif est d'entraîner un réseau pour jouer au jeu de Go.  
- Afin de garantir une équité en termes de ressources d'entraînement, le nombre de paramètres des réseaux soumis doit être inférieur à 100 000.  
- Le nombre maximal d'étudiants par équipe est de deux.  
- Les données utilisées pour l'entraînement proviennent des parties auto-jouées du programme Katago Go.  
- Le jeu de données d'entraînement contient un total de 1 000 000 de parties différentes.  
- Les données d'entrée sont composées de 31 plans de taille 19x19 :  
  - Couleur au trait  
  - Échelles  
  - État actuel sur deux plans  
  - Deux états précédents sur plusieurs plans  
- Les cibles de sortie sont :  
  - **La politique** : un vecteur de taille 361 avec `1.0` pour le coup joué, `0.0` pour les autres coups.  
  - **La valeur** : une valeur entre `0.0` et `1.0` fournie par la recherche d'arbre Monte-Carlo, représentant la probabilité de victoire de Blanc.

- Le projet a été écrit et fonctionne sous Ubuntu 22.04.  
- Il utilise TensorFlow 2.9 et Keras pour le réseau.  
- Un exemple de réseau convolutionnel avec deux têtes est donné dans le fichier `golois.py` et est sauvegardé dans le fichier `test.h5`.  
- Les réseaux que vous concevez et entraînez doivent également avoir les mêmes têtes de politique et de valeur et être sauvegardés au format `.h5`.  
- Un exemple de réseau et un épisode d'entraînement sont fournis dans le fichier `golois.py`.  
- Si vous souhaitez compiler la bibliothèque Golois, vous devez installer **Pybind11** et exécuter `compile.sh`.

## Tournois

- Toutes les deux semaines environ, j'organiserai un tournoi entre les réseaux que vous téléchargez.  
- Chaque nom de réseau correspond aux noms des étudiants qui ont conçu et entraîné le réseau.  
- Le modèle doit être sauvegardé au format **Keras h5**.  
- Un tournoi en **round robin** sera organisé et les résultats seront envoyés par e-mail.  
- Chaque réseau sera utilisé par un moteur **PUCT**, qui disposera de **2 secondes de temps CPU** par coup pour jouer dans le tournoi.

## Exemple de réseau

```python
planes = 31
moves = 361
N = 10000
epochs = 20
batch = 128
filters = 32
input_data = np.random.randint(2, size=(N, 19, 19, planes))
input_data = input_data.astype ('float32')
policy = np.random.randint(moves, size=(N,))
policy = keras.utils.to_categorical (policy)
value = np.random.randint(2, size=(N,))
value = value.astype ('float32')
end = np.random.randint(2, size=(N, 19, 19, 2))
end = end.astype ('float32')
groups = np.zeros((N, 19, 19, 1))
groups = groups.astype ('float32')

input = keras.Input(shape=(19, 19, planes), name='board')
x = layers.Conv2D(filters, 1, activation='relu', padding='same')(input)
for i in range (5):
  x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
policy_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(x)
policy_head = layers.Flatten()(policy_head)
policy_head = layers.Activation('softmax', name='policy')(policy_head)
value_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(x)
value_head = layers.Flatten()(value_head)
value_head = layers.Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(value_head)
value_head = layers.Dense(1, activation='sigmoid', name='value', kernel_regularizer=regularizers.l2(0.0001))(value_head)
model = keras.Model(inputs=input, outputs=[policy_head, value_head])

model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.0005, momentum=0.9),
loss={'policy': 'categorical_crossentropy', 'value': 'binary_crossentropy'},
loss_weights={'policy' : 1.0, 'value' : 1.0},
metrics={'policy': 'categorical_accuracy', 'value': 'mse'})
for i in range (1, epochs + 1):
  print ('epoch ' + str (i))
  golois.getBatch (input_data, policy, value, end, groups, i * N)
  history = model.fit(input_data,
  {'policy': policy, 'value': value},
  epochs=1, batch_size=batch)
  if (i % 5 == 0):
  gc.collect ()
  if (i % 20 == 0):
  golois.getValidation (input_data, policy, value, end)
  val = model.evaluate (input_data,
  [policy, value], verbose = 0, batch_size=batch)
  print ("val =", val)
  model.save ('test.h5')
```

## Instructions :  
- Entraînez un réseau pour jouer au Go.  
- Soumettez les réseaux entraînés **avant samedi soir**.  
- Tournoi des réseaux **chaque dimanche**.  
- Téléchargez un réseau **avant la fin de la session**.

In [ ]:
!wget https://www.lamsade.dauphine.fr/~cazenave/project2025.zip
!unzip project2025.zip
!ls -l

In [ ]:
!pip install tensorrt-bindings==8.6.1
!pip install --extra-index-url https://pypi.nvidia.com tensorrt-libs
!pip install tensorflow[and-cuda]==2.15.0

## Importation des librairies

In [ ]:
from abc import ABC, abstractmethod
from google.colab import files
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import regularizers
import gc
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

import golois

## Création de la classe abstraite `GONet`

In [ ]:
class GONet(ABC):

    def __init__(
        self, batch=128, epochs=20, filters=32, moves=361, N=10000, planes=31
    ):
        self.batch = batch
        self.epochs = epochs
        self.filters = filters
        self.moves = moves
        self.N = N
        self.planes = planes
        self.set_input_data(N, planes)
        self.set_policy(N, moves)
        self.set_value(N)
        self.set_end(N)
        self.set_groups(N)
        print("Tensorflow version", tf.__version__)
        print("getValidation", flush=True)
        golois.getValidation(
            self.input_data, self.policy, self.value, self.end
        )
        self.loss_total = []
        self.policy_loss  = []
        self.value_loss  = []
        self.policy_acc = []
        self.value_mse = []

    def set_input_data(self, N, planes):
        input_data = np.random.randint(2, size=(N, 19, 19, planes))
        self.input_data = input_data.astype("float32")

    def get_input_data(self):
        return self.input_data

    def set_policy(self, N, moves):
        policy = np.random.randint(moves, size=(N,))
        self.policy = keras.utils.to_categorical(policy)

    def get_policy(self):
        return self.policy

    def set_value(self, N):
        value = np.random.randint(2, size=(N,))
        self.value = value.astype("float32")

    def get_value(self):
        return self.value

    def set_end(self, N):
        end = np.random.randint(2, size=(N, 19, 19, 2))
        self.end = end.astype("float32")

    def get_end(self):
        return self.end

    def set_groups(self, N):
        groups = np.zeros((N, 19, 19, 1))
        self.groups = groups.astype("float32")

    def get_groups(self):
        return self.groups

    @abstractmethod
    def set_model(self):
        pass

    @abstractmethod
    def create_policy_value_heads(self):
        """
        Fonction de création des en-têtes de politique et de valeur.
        """
        pass

    def train_model(self, epochs):
        for i in range(1, epochs + 1):
            print(f"epoch {str(i)}")
            golois.getBatch(
                self.input_data,
                self.policy,
                self.value,
                self.end,
                self.groups,
                i * self.N,
            )
            history = self.model.fit(
                self.input_data,
                {"policy": self.policy, "value": self.value},
                epochs=1,
                batch_size=self.batch,
            )
            # Extraction des valeurs depuis history.history
            self.loss_total.append(history.history["loss"][0])  # Loss globale
            self.policy_loss.append(history.history["policy_loss"][0])  # Policy loss
            self.value_loss.append(history.history["value_loss"][0])  # Value loss
            self.policy_acc.append(history.history["policy_categorical_accuracy"][0])  # Policy accuracy
            self.value_mse.append(history.history["value_mse"][0])  # Value MSE
            if i % 5 == 0:
                gc.collect()
            if i % 20 == 0:
                golois.getValidation(
                    self.input_data, self.policy, self.value, self.end
                )
                val = self.model.evaluate(
                    self.input_data,
                    [self.policy, self.value],
                    verbose=0,
                    batch_size=self.batch,
                )
                print(f"{val=}")

    def save_model(self, name):
        self.model.save(name)
        files.download(name)

    def plot_training_history(self):
        """Trace les courbes des pertes et métriques en fonction des époques."""
        epochs = range(1, len(self.loss_total) + 1)  # Liste des époques

        plt.figure(figsize=(12, 6))

        # Graphique des pertes
        plt.subplot(1, 2, 1)
        plt.plot(epochs, self.loss_total, label="Loss totale", marker="o")
        plt.plot(epochs, self.policy_loss, label="Policy Loss", marker="o")
        plt.plot(epochs, self.value_loss, label="Value Loss", marker="o")
        plt.xlabel("Époques")
        plt.ylabel("Valeur de la perte")
        plt.title("Évolution des pertes")
        plt.legend()
        plt.grid(True)

        # Graphique des métriques
        plt.subplot(1, 2, 2)
        plt.plot(epochs, self.policy_acc, label="Policy Accuracy", marker="o")
        plt.plot(epochs, self.value_mse, label="Value MSE", marker="o")
        plt.xlabel("Époques")
        plt.ylabel("Valeur des métriques")
        plt.title("Évolution des métriques")
        plt.legend()
        plt.grid(True)

        # Affichage
        plt.tight_layout()
        plt.show()

    def log_accuracy(self, results_dict={}):
        """Logs the model’s last accuracy into the given dictionary."""
        results_dict[self.__class__.__name__] = {
          "instance": self,
          "accuracy": self.policy_acc[-1]
        }

In [ ]:
def save_best_model(results_dict, model_name="test.h5"):
    """
    Returns the class instance with the highest accuracy from the dictionary.
    """
    if not results_dict:
        return None  # Handle empty dictionary case

    # Find the key with the max accuracy
    best_model_key = max(results_dict, key=lambda k: results_dict[k]["accuracy"])

    # Retrieve the best accuracy
    best_accuracy = results_dict[best_model_key]["accuracy"]

    # Retrieve the best instance
    best_instance = results_dict[best_model_key]["instance"]

    print(f"Le réseau {best_model_key} est celui qui a enregistré la meilleur accuracy : {best_accuracy}")

    best_instance.save_model(model_name)

Nous allons définir un dictionnaire intitulé `log_accuracy_dict` qui contiendra les *accuracies* successives pour chaque réseau. Ce dernier constituera un historique des performances des réseaux entraînés.

In [ ]:
log_accuracy_dict = {}

Création de la classe `GONetDemo`

In [ ]:
class GONetDemo(GONet):

    def __init__(
        self, batch=128, epochs=20, filters=32, moves=361, N=10000, planes=31
    ):
        super().__init__(batch, epochs, filters, moves, N, planes)

    def create_policy_value_heads(self, x, input_layer):
        """
        Fonction de création des en-têtes de politique et de valeur.
        """
        policy_head = layers.Conv2D(
            1,
            1,
            activation="relu",
            padding="same",
            use_bias=False,
            kernel_regularizer=regularizers.l2(0.0001),
        )(x)
        policy_head = layers.Flatten()(policy_head)
        policy_head = layers.Activation("softmax", name="policy")(policy_head)
        value_head = layers.Conv2D(
            1,
            1,
            activation="relu",
            padding="same",
            use_bias=False,
            kernel_regularizer=regularizers.l2(0.0001),
        )(x)
        value_head = layers.Flatten()(value_head)
        value_head = layers.Dense(
            50, activation="relu", kernel_regularizer=regularizers.l2(0.0001)
        )(value_head)
        value_head = layers.Dense(
            1,
            activation="sigmoid",
            name="value",
            kernel_regularizer=regularizers.l2(0.0001),
        )(value_head)

        model = keras.Model(inputs=input_layer, outputs=[policy_head, value_head])

        model.summary()

        model.compile(
            optimizer=keras.optimizers.SGD(learning_rate=0.0005, momentum=0.9),
            loss={
                "policy": "categorical_crossentropy",
                "value": "binary_crossentropy",
            },
            loss_weights={"policy": 1.0, "value": 1.0},
            metrics={"policy": "categorical_accuracy", "value": "mse"},
        )
        return model

    def set_model(self):
        input_layer = keras.Input(shape=(19, 19, self.planes), name="board")
        x = layers.Conv2D(self.filters, 1, activation="relu", padding="same")(
            input_layer
        )
        for _ in range(5):
            x = layers.Conv2D(
                self.filters, 3, activation="relu", padding="same"
            )(x)
        self.model = self.create_policy_value_heads(x, input_layer)

In [ ]:
GONetDemo_instance = GONetDemo()
GONetDemo_instance.set_model()
GONetDemo_instance.train_model(20)
GONetDemo_instance.plot_training_history()
GONetDemo_instance.log_accuracy(log_accuracy_dict)

Création de la classe `GONetRes_cnn`

In [ ]:
class GONetRes_cnn(GONet):
    def __init__(
        self, batch=128, epochs=20, filters=32, moves=361, N=10000, planes=31
    ):
        super().__init__(batch, epochs, filters, moves, N, planes)

    def create_policy_value_heads(self, x, input_layer):
        """
        Fonction de création des en-têtes de politique et de valeur.
        """
        policy_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias=False)(x)
        policy_head = layers.Flatten()(policy_head)
        policy_head = layers.Activation('softmax', name='policy')(policy_head)

        value_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias=False)(x)
        value_head = layers.Flatten()(value_head)
        value_head = layers.Dense(50, activation='relu')(value_head)
        value_head = layers.Dense(1, activation='sigmoid', name='value')(value_head)

        model = models.Model(inputs=input_layer, outputs=[policy_head, value_head])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
                      loss={'policy': 'categorical_crossentropy', 'value': 'binary_crossentropy'},
                      metrics={'policy': 'categorical_accuracy', 'value': 'mse'})
        return model

    def set_model(self, filters=32, planes=31):
        input_layer = layers.Input(shape=(19, 19, planes), name='board')
        x = layers.Conv2D(filters, 1, activation='relu', padding='same')(input_layer)
        for _ in range(5):
            x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
        self.model = self.create_policy_value_heads(x, input_layer)

In [ ]:
GONetRes_cnn_instance = GONetRes_cnn()
GONetRes_cnn_instance.set_model()
GONetRes_cnn_instance.train_model(20)
GONetRes_cnn_instance.plot_training_history()
GONetRes_cnn_instance.log_accuracy(log_accuracy_dict)

## Création de la classe `GONetRes_resnet`

In [ ]:
class GONetRes_resnet(GONet):
    def __init__(
        self, batch=128, epochs=20, filters=32, moves=361, N=10000, planes=31
    ):
        super().__init__(batch, epochs, filters, moves, N, planes)

    def create_policy_value_heads(self, x, input_layer):
        """
        Fonction de création des en-têtes de politique et de valeur.
        """
        policy_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias=False)(x)
        policy_head = layers.Flatten()(policy_head)
        policy_head = layers.Activation('softmax', name='policy')(policy_head)

        value_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias=False)(x)
        value_head = layers.Flatten()(value_head)
        value_head = layers.Dense(50, activation='relu')(value_head)
        value_head = layers.Dense(1, activation='sigmoid', name='value')(value_head)

        model = models.Model(inputs=input_layer, outputs=[policy_head, value_head])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
                      loss={'policy': 'categorical_crossentropy', 'value': 'binary_crossentropy'},
                      metrics={'policy': 'categorical_accuracy', 'value': 'mse'})
        return model

    def residual_block(self, x, filters):
        shortcut = x
        x = layers.Conv2D(filters, (3,3), padding='same', use_bias=False)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        x = layers.Conv2D(filters, (3,3), padding='same', use_bias=False)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Add()([x, shortcut])
        x = layers.ReLU()(x)
        return x

    def set_model(self, filters=64, planes=31):
        input_layer = layers.Input(shape=(19, 19, planes), name='board')
        x = layers.Conv2D(filters, 3, activation='relu', padding='same')(input_layer)
        for _ in range(5):
            x = self.residual_block(x, filters)
        self.model = self.create_policy_value_heads(x, input_layer)

In [ ]:
GONetRes_resnet_instance = GONetRes_resnet()
GONetRes_resnet_instance.set_model()
GONetRes_resnet_instance.train_model(20)
GONetRes_resnet_instance.plot_training_history()
GONetRes_resnet_instance.log_accuracy(log_accuracy_dict)

In [ ]:
save_best_model(log_accuracy_dict)